In [1]:
! pip install transformers
! pip install simpletransformers

     |████████████████████████████████| 3.5 MB 8.1 MB/s 
     |████████████████████████████████| 6.8 MB 35.5 MB/s 
     |████████████████████████████████| 67 kB 3.8 MB/s 
     |████████████████████████████████| 895 kB 57.2 MB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 248 kB 9.8 MB/s 
     |████████████████████████████████| 1.2 MB 69.4 MB/s 
     |████████████████████████████████| 1.7 MB 62.8 MB/s 
     |████████████████████████████████| 9.7 MB 49.6 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 311 kB 79.3 MB/s 
     |████████████████████████████████| 144 kB 87.3 MB/s 
     |████████████████████████████████| 180 kB 82.1 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 133 kB 

In [2]:
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd
from google.colab import files

model_args = NERArgs(learning_rate=1e-5, num_train_epochs=4, eval_batch_size=8, train_batch_size=8, weight_decay=1)
model_args.labels_list = ["B-LOC", "I-LOC", "B-ORG", "I-ORG", "B-DATE", "B-PER", "I-PER", "I-DATE", "O"]


model = NERModel(
    "xlmroberta", "xlm-roberta-large", 
    args=model_args,
)

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [3]:
uploaded = files.upload()

Saving train_swa.txt to train_swa.txt


In [4]:
def load_train():
  df_text = uploaded['train_swa.txt'].decode("utf-8")
  text = df_text.split('\n\n')

  df_train = pd.DataFrame()
  df_train['sentence_id'] = pd.Series()
  df_train['words'] = pd.Series()
  df_train['labels'] = pd.Series()

  sentence_num = []
  words = []
  tag = []
  for i in range(len(text)):
      sentence = text[i].split('\n')
      for j in range(len(sentence)-1):
          word = sentence[j].split(" ")
          sentence_num.append(i)
          words.append(word[0])
          tag.append(word[1])
     
  df_train['sentence_id'] = sentence_num
  df_train['words'] = words
  df_train['labels'] = tag
  return df_train

In [5]:
df_train1 = load_train()
df_train1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,sentence_id,words,labels
0,0,Wizara,B-ORG
1,0,ya,I-ORG
2,0,afya,I-ORG
3,0,ya,I-ORG
4,0,Tanzania,I-ORG
...,...,...,...
54485,2108,na,O
54486,2108,yale,O
54487,2108,aliyoyafanya,O
54488,2108,mtoto,O


In [6]:
df_train = df_train1.head(5000)
df_test = df_train1.tail(5000)

In [9]:
model.train_model(df_train)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 4:   0%|          | 0/24 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/24 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/24 [00:00<?, ?it/s]

(96, 0.5684919273480773)

In [7]:
uploaded = files.upload()

Saving swa_mat_acts.txt to swa_mat_acts.txt


In [8]:
test_text = uploaded['swa_mat_acts.txt'].decode("utf-8")
test_text = "".join(filter(lambda x: not x.isdigit(), test_text))
text = test_text.split('\n')
new_text = [s.replace("\x0c", "") for s in text]
new_text = [s.replace("\t", "") for s in new_text]
new_text

['Mathayo ',
 '',
 ' Mathayo',
 '',
 ' Kumbukumbu Za Ukoo Wa Yesu Kristo',
 '',
 ' (Luka :-)',
 '',
 '  Habari za ukoo wa Yesu Kristo mwana wa Daudi, mwana wa Abrahamu:',
 ' ',
 '   Abrahamu akamzaa Isaki,',
 ' Isaki akamzaa Yakobo,',
 ' Yakobo akawazaa Yuda na ndugu zake,',
 '   Yuda akawazaa Peresi na Zera, ambao mama yao alikuwa Tamari,',
 ' Peresi akamzaa Hesroni,',
 ' Hesroni akamzaa Aramu,',
 '   Aramu akamzaa Aminadabu,',
 ' Aminadabu akamzaa Nashoni,',
 ' Nashoni akamzaa Salmoni,',
 '   Salmoni akamzaa Boazi, na mama yake Boazi alikuwa Rahabu,',
 ' Boazi akamzaa Obedi, ambaye mama yake alikuwa Ruthu,',
 ' Obedi akamzaa Yese,',
 '   Yese akamzaa Daudi ambaye alikuwa mfalme.',
 ' ',
 ' Daudi akamzaa Solomoni, ambaye mama yake ni yule aliyekuwa mke wa Uria.',
 '   Solomoni akamzaa Rehoboamu,',
 ' Rehoboamu akamzaa Abiya,',
 ' Abiya akamzaa Asa,',
 '   Asa akamzaa Yehoshafati,',
 ' Yehoshafati akamzaa Yoramu,',
 ' Yoramu akamzaa Uzia,',
 '   Uzia akamzaa Yothamu,',
 ' Yothamu akamz

In [10]:
predictions, raw_outputs = model.predict(new_text)

  0%|          | 0/4 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/274 [00:00<?, ?it/s]

In [11]:
def new_data():
  id = len(df_train)

  keys = []
  values = []
  sentence_id = []
  for idx in range(len(predictions)):
    for length in range(len(predictions[idx])):
      for key in predictions[idx][length].keys():
        keys.append(key)
        sentence_id.append(id)
      for value in predictions[idx][length].values():
        values.append(value)

    id += 1 

  df_pred = pd.DataFrame({'sentence_id':sentence_id, 'words':keys, 'labels':values})
  df_newTrain = pd.concat([df_train, df_pred])
  df_newTrain.reset_index(inplace=True, drop=True)
  return df_newTrain

In [12]:
df_newTrain = new_data()
df_newTrain

,sentence_id,words,labels
0,0,Wizara,B-ORG
1,0,ya,I-ORG
2,0,afya,I-ORG
3,0,ya,I-ORG
4,0,Tanzania,I-ORG
...,...,...,...
39653,7189,wote,O
39654,7189,na,O
39655,7189,bila,O
39656,7189,kizuizi,O


In [13]:
for epoch in range(75):

  print(f"Epoch {epoch+1}")
  model_args = NERArgs(learning_rate=1e-5, num_train_epochs=1, eval_batch_size=8, train_batch_size=8, weight_decay=1, manual_seed=42, overwrite_output_dir=True)
  model_args.labels_list = ["B-LOC", "I-LOC", "B-ORG", "I-ORG", "B-DATE", "B-PER", "I-PER", "I-DATE", "O"]

  model = NERModel(
      "xlmroberta", "/content/outputs/", 
      args=model_args,
  )

  model.train_model(df_newTrain)
  predictions, raw_outputs = model.predict(text)
  df_newTrain = new_data()

Epoch 1


  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/215 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/215 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/274 [00:00<?, ?it/s]

In [14]:
result, model_outputs, wrong_preds = model.eval_model(df_test)
result

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

{'eval_loss': 0.5563305951654911,
 'f1_score': 0.8317535545023697,
 'precision': 0.8125,
 'recall': 0.8519417475728155}